In [61]:
import random
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

device = torch.device('cuda:0')

In [85]:
data = pd.read_csv('wine.data', header=0)
train_size = int(0.8 * len(data))
y_data = data['class'].values
# need to convert 1,2,3 to 100, 010, 001
x_data = data[data.columns.difference(['class'])].values
test_size = len(data) - train_size
x_train, x_test = torch.utils.data.random_split(x_data, [train_size, test_size])
y_train, y_test = torch.utils.data.random_split(y_data, [train_size, test_size])
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [79]:
class WineNet(torch.nn.ModuleList):
    def __init__(self, n_hid_neurons):
        super(WineNet, self).__init__()
        
        self.fc1 = torch.nn.Linear(13, n_hid_neurons)
        self.act1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_hid_neurons, n_hid_neurons)
        self.act2 = torch.nn.Sigmoid()
        self.fc3 = torch.nn.Linear(n_hid_neurons, 3)
        self.sm = torch.nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.fc3(x)
        return x
    
    def inference(self, x):
        x = self.forward(x)
        x = self.sm(x)
        return x
    
winenet = WineNet(5)

In [80]:
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(winenet.parameters(), lr=1e-3)

In [84]:
batch_size = 10
epochs = range(5000)

for epoch in epochs:
    order = np.random.permutation(len(x_train))
    for start_index in range(0, len(x_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index + batch_size]
        x_batch = x_train[batch_indexes]
        y_batch = y_train[batch_indexes]
        
        preds = winenet.forward(x_batch)
        print(preds, y_batch)
        loss_val = loss_func(preds, y_batch)
        loss_val.backward()
        optimizer.step()
    if epoch % 100 == 0:
        test_pred = winenet.forward(x_test)
        test_preds = test_preds.argmax(dim=1)
        print((test_preds == y_test).float().mean())

tensor([[ 0.1153, -0.3190,  0.6081],
        [ 0.1153, -0.3190,  0.6081],
        [ 0.1153, -0.3190,  0.6081],
        [ 0.1153, -0.3190,  0.6081],
        [ 0.1153, -0.3190,  0.6081],
        [ 0.1153, -0.3190,  0.6081],
        [ 0.1153, -0.3190,  0.6081],
        [ 0.1153, -0.3190,  0.6081],
        [ 0.1153, -0.3190,  0.6081],
        [ 0.1153, -0.3190,  0.6081]], grad_fn=<AddmmBackward>) tensor([1, 1, 1, 1, 1, 3, 3, 1, 1, 1])


RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at C:\w\1\s\tmp_conda_3.7_055457\conda\conda-bld\pytorch_1565416617654\work\aten\src\THNN/generic/ClassNLLCriterion.c:94